In [1]:
import pickle as pkl
import re 

path = '/home/liuxiao/TuringGithub/XiaoData/ZeroShotJudge/chatgpt_object_detection_zeroshot_judge.pkl'
with open(path,'rb') as f:
    judge_results = pkl.load(f)

In [2]:
judge_results

{'2276.jpg': '**1. human face, person, wrinkle**  \n**Classification: Human**  \n**Justification:** The inclusion of “wrinkle” is a subtle and detailed observation that reflects an understanding of age-related features typically not prioritized by AI models, which focus more on general object categories.\n\n**2. face, person, human**  \n**Classification: AI-generated**  \n**Justification:** This set is somewhat redundant — “face” and “person” are common in AI outputs, and the addition of “human” feels tautological or overly simplistic, which is typical of AI models attempting to be exhaustive but not nuanced.\n\n**3. person, tie**  \n**Classification: AI-generated**  \n**Justification:** This identification is minimal and functional, representing the most prominent objects (the man and his tie) in a way that is typical of AI systems focused on concrete, detectable features.\n\n**4. photo, album, room**  \n**Classification: AI-generated**  \n**Justification:** These labels do not corres

In [9]:
import re

# special case '0525.jpg'

def findanswer(text):
#     templates = [r'(?:\*\*Caption\s*(\d)|^(\d)\. \:)',
#                  r'(?:\#\#\#\sCaption\s*(\d)|^(\d)\. \:)',
#                  r'(?:\*\**(\d)|^(\d)\. \:)',
#                  r'(?:\#\#\#\s(\d)|^(\d)\. \:)',
#                  r'(?:(\d)\. \*\*)']
    results = []
#     print(re.search(templates[0],text))
    templates = [r'(?:.{0,6}Summary of Classifications)']
    flag = False
    for template in templates:
        if re.search(template,text) is not None:
            flag = True
            break
    if not flag:
        print(text)
        return        
    
#     print(re.search(template,text).start())
    summary_text = text[re.search(template,text).start():].replace('Human or AI-generated result classifications','')
    summary_text = summary_text.replace('Human or AI Classification Results:','')
    summary_text = summary_text.replace('Human or AI','')
    
    results = []
    for i, m in enumerate(re.finditer('(?:(AI-generated)|(Human))',summary_text)):
        print(m.group(0))
        if 'AI-generated' in m.group(0):
            results.append('M')
        elif 'Human' in m.group(0):
            results.append('H')
        else:
            assert False
        if i == 5:
            break

    if i==1:
        summary = summary_text.split('AI-generated:')
        if len(summary)!=2:
            assert False
        results = [0] * 5
        for pos in re.finditer('\d',summary[0]):
#             print(pos.group(0))
            results[int(pos.group(0))-1] = 'H'
        for pos in re.finditer('\d',summary[1]):
#             print(pos)
            results[int(pos.group(0))-1] = 'M'        
#         print(results)
#         print(summary_text)
    
    elif i!=4:
        assert False
    if len(results) != 5 or len([_ for _ in results if _ in ['M','H']]) != 5:
        assert False
    return results
    
results_d = {'74.jpg':['H','M','M','M','M']}
for k,text in judge_results.items():
    
    print('==='*20)
    print(k)
    if k not in results_d:
        results = findanswer(text)
        results_d[k] = results

2276.jpg
Human
AI-generated
AI-generated
AI-generated
Human
672.jpg
AI-generated
AI-generated
Human
AI-generated
Human
191.jpg
AI-generated
Human
AI-generated
Human
AI-generated
380.jpg
Human
AI-generated
Human
AI-generated
AI-generated
692.jpg
Human
AI-generated
AI-generated
AI-generated
Human
995.jpg
Human
AI-generated
AI-generated
AI-generated
Human
409.jpg
Human
AI-generated
Human
AI-generated
AI-generated
588.jpg
Human
AI-generated
AI-generated
Human
AI-generated
805.jpg
AI-generated
AI-generated
AI-generated
Human
Human
253.jpg
Human
AI-generated
AI-generated
Human
AI-generated
926.jpg
Human
AI-generated
AI-generated
Human
Human
365.jpg
Human
AI-generated
Human
AI-generated
AI-generated
756.jpg
Human
AI-generated
AI-generated
Human
Human
824.jpg
Human
AI-generated
AI-generated
AI-generated
Human
224.jpg
Human
AI-generated
AI-generated
AI-generated
Human
565.jpg
AI-generated
AI-generated
AI-generated
Human
Human
455.jpg
AI-generated
AI-generated
Human
Human
AI-generated
923.jpg
AI

AI-generated
624.jpg
AI-generated
Human
AI-generated
AI-generated
Human
139.jpg
Human
AI-generated
Human
AI-generated
Human
862.jpg
AI-generated
Human
Human
AI-generated
AI-generated
774.jpg
Human
AI-generated
Human
AI-generated
AI-generated
720.jpg
Human
AI-generated
AI-generated
Human
Human
160.jpg
Human
AI-generated
Human
AI-generated
Human
3087.jpg
AI-generated
AI-generated
Human
AI-generated
Human
361.jpg
Human
AI-generated
AI-generated
AI-generated
Human
117.jpg
AI-generated
AI-generated
Human
AI-generated
Human
121.jpg
AI-generated
Human
AI-generated
Human
AI-generated
2614.jpg
Human
AI-generated
Human
AI-generated
Human
3269.jpg
Human
AI-generated
AI-generated
AI-generated
Human
248.jpg
Human
AI-generated
AI-generated
Human
AI-generated
651.jpg
Human
Human
AI-generated
AI-generated
AI-generated
352.jpg
AI-generated
Human
AI-generated
AI-generated
Human
4418.jpg
AI-generated
Human
AI-generated
Human
AI-generated
772.jpg
Human
AI-generated
Human
AI-generated
AI-generated
946.jpg


### Compile Trials

In [10]:
import json
import numpy as np
path = '/home/liuxiao/TuringGithub/XiaoData/ZeroShotJudge/groundtruths_d_objdet.json'
with open(path) as f:
    groundtruths_d_objdet = json.load(f)

# templates    
# Trial_1': {'trialData': {'response_speaker': 'human',
#    'groundtruth': 'human',
#    'machine_groundtruth': ''}}
ai_judge = {}

In [11]:
groundtruths_d_objdet

{'2276.jpg': [{'imageID': '2276.jpg',
   'response': 'human face, person, wrinkle',
   'groundtruth': 'machine',
   'human_groundtruth': '',
   'machine_groundtruth': 'azure'},
  {'imageID': '2276.jpg',
   'response': 'face, person, human',
   'groundtruth': 'machine',
   'human_groundtruth': '',
   'machine_groundtruth': 'rekognition'},
  {'imageID': '2276.jpg',
   'response': 'person, tie',
   'groundtruth': 'machine',
   'human_groundtruth': '',
   'machine_groundtruth': 'detectron'},
  {'imageID': '2276.jpg',
   'response': 'photo, album, room',
   'groundtruth': 'human',
   'human_groundtruth': '',
   'machine_groundtruth': ''},
  {'imageID': '2276.jpg',
   'response': 'forehead, nose, hair',
   'groundtruth': 'machine',
   'human_groundtruth': '',
   'machine_groundtruth': 'google'}],
 '672.jpg': [{'imageID': '672.jpg',
   'response': 'fire hydrant, car, bird',
   'groundtruth': 'machine',
   'human_groundtruth': '',
   'machine_groundtruth': 'detectron'},
  {'imageID': '672.jpg'

In [12]:
trial_count = 1
# label_dict = {'human':0,'machine':1,'blip':1,'microsoft_api':1,'ofa':1,'git':1,'clipcap':1,'catch':1}
label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'detectron':1,'google':1,'catch':1,'rekognition':1}
def construct_confmat():
    labels = ['h_azure','h_detectron','h_google','h_rekognition']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

confmat_overall = construct_confmat()
for k, gt_d in groundtruths_d_objdet.items():
    for i, gt in enumerate(gt_d):
        if k == '548.jpg': continue
        trialdata = {'response_speaker':{'H':'human','M':'machine'}[results_d[k][i]],
                    'groundtruth':gt['groundtruth'],
                    'machine_groundtruth':gt['machine_groundtruth']}
        ai_judge['Trial_{}'.format(trial_count)] = {'trialData':trialdata}
        trial_count += 1
        
        confmat_overall['h_machine'][label_dict[trialdata['groundtruth']]][label_dict[trialdata['response_speaker']]] += 1
        
        if trialdata['groundtruth'] == 'machine':
            confmat_overall['h_{}'.format(trialdata['machine_groundtruth'])][0][label_dict[trialdata['response_speaker']]] += 1          
            
        

In [13]:
confmat_overall

{'h_machine': array([[ 473.,  334.],
        [1133., 2095.]]),
 'h_azure': array([[290., 517.]]),
 'h_detectron': array([[410., 397.]]),
 'h_google': array([[214., 593.]]),
 'h_rekognition': array([[219., 588.]])}

In [14]:
json.dump(ai_judge, open("/home/liuxiao/TuringGithubRunnable/multi_label_prediction/Plot/Data_all/ai_judge_zeroshot_chatgpt_objdet.json","w"))
json.dump(ai_judge, open("/home/liuxiao/TuringGithub/XiaoData/all_results/ai_judge_zeroshot_chatgpt_objdet.json","w"))


In [16]:
trialdata

{'response_speaker': 'machine',
 'groundtruth': 'machine',
 'machine_groundtruth': 'google'}

Bad pipe message: %s [b'O?\xe7s\x88\x85\x93\xa5*}\x1c\x9a\xf5rl\x91w:\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j']
Bad pipe message: %s [b'\x16\'\x8by\x85(1N\xa0%\xb58\xb7\xae\xb9\xde\xa8#\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\